# 1. Write some documentation

The data are the latest transactions of cryptocurrencies happened in major exchange websites and the metadata of the exchange websites. 

how it was collected: from the FAQ section learned that they collect data through API of their data sources, so not sure about the origin source.

link to the API's documentation: https://docs.coinapi.io/?python#introduction

link to the specific endpoint(s): 'https://rest.coinapi.io/v1/exchanges' & 'https://rest.coinapi.io/v1/trades/latest'

Structure: it's a list of dictionaries

## Test request

In [120]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import json
pd.set_option('display.max_rows', 200)

In [121]:
#TODO: fill api key
api_key = "CEABFBD9-2B2A-4EAE-A771-5250E2CD64D6"
#no parameters in this case
url = 'https://rest.coinapi.io/v1/exchanges'
headers = {'X-CoinAPI-Key' : api_key}
res = requests.get(url, headers=headers)
dom = BeautifulSoup(res.text, 'lxml')

In [122]:
json.loads(dom.string)[0]

{'exchange_id': 'OKCOIN_CNY',
 'website': 'https://www.okcoin.cn/',
 'name': 'OKCoin CNY',
 'data_start': '2013-06-12',
 'data_end': '2018-03-09',
 'data_quote_start': '2015-02-15T12:53:50.3430000Z',
 'data_quote_end': '2018-03-09T23:34:52.5800000Z',
 'data_orderbook_start': '2015-02-15T12:53:50.3430000Z',
 'data_orderbook_end': '2018-03-09T23:34:52.5800000Z',
 'data_trade_start': '2013-06-12T14:24:24.0000000Z',
 'data_trade_end': '2017-11-01T16:30:39.7077259Z',
 'data_symbols_count': 2,
 'volume_1hrs_usd': 0.0,
 'volume_1day_usd': 0.0,
 'volume_1mth_usd': 0.0}

# 2. Retrieve the data, create a DataFrame

In [123]:
pd.set_option('display.max_rows', 300)
exchange_web_df = pd.DataFrame(json.loads(dom.string))

In [124]:
trades_url = 'https://rest.coinapi.io/v1/trades/latest'
trade_res = requests.get(trades_url, headers=headers)

In [125]:
dom2 = BeautifulSoup(trade_res.text, 'lxml')
trades_df = pd.DataFrame(json.loads(dom2.string))

## Attempt to merge two data frames to come up the latest transaction amount and type in exchange websites

merge the two DataFrames and show certain column values

In [126]:
trades_df['exchange_id'] = trades_df['symbol_id'].str.extract('([^_]*)')
combine = pd.merge(exchange_web_df, trades_df, on='exchange_id', how='right').reindex(columns=['symbol_id', 'price', 'size', 'taker_side', 'volume_1hrs_usd'])
combine['total'] = combine['price']

group the price column by the symbol_id column then sum the price up for each symbol_id

In [127]:

combine['price'].groupby(combine['symbol_id']).sum()

symbol_id
BIBOX_SPOT_4BTC_USDT        26351.400000
BIBOX_SPOT_4EOS_USDT            7.282000
BIBOX_SPOT_BAT_BIX              1.596307
BIBOX_SPOT_BIX_USDT             0.140200
BIBOX_SPOT_BTC_USDT         61507.674200
BIBOX_SPOT_DDAM_ETH             0.001065
BIBOX_SPOT_EOS_ETH              0.016105
BIBOX_SPOT_EOS_USDT            10.923700
BIBOX_SPOT_ETC_USDT            16.766500
BIBOX_SPOT_ETH_BTC              0.025732
BIBOX_SPOT_NEO_BTC              0.001348
BIBOX_SPOT_ONT_BIX              5.087261
BIBOX_SPOT_ONT_USDT             0.712900
BIBOX_SPOT_RED_ETH              0.000104
BIBOX_SPOT_TRX_ETH              0.000077
BIBOX_SPOT_XRP_BTC              0.000027
BINANCEFTS_PERP_BTC_USDT     8790.360000
BINANCE_SPOT_BNB_ETH            0.088024
BINANCE_SPOT_BTC_BUSD        8787.140000
BINANCE_SPOT_BTC_USDT       17569.410000
BINANCE_SPOT_BUSD_USDT          0.999700
BINANCE_SPOT_HBAR_BTC           0.000005
BINANCE_SPOT_TRX_USDC           0.017370
BINANCE_SPOT_ZIL_BNB            0.000308
BINANC